pca_glove_50features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/glove_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-0.073512,-3.579602,2.542294,0.829520,-1.309279,-2.129783,3.003700,-1.607696,-0.377434,...,0.451810,0.450999,-0.897498,-0.197427,-0.556504,-1.252244,-0.218438,-0.182912,0.638235,-0.966327
1,1,9.926291,11.637135,-6.480039,-7.815047,-5.385470,9.829159,5.992433,-3.211267,-3.751414,...,-0.767886,-0.963754,-0.291192,-4.728044,0.295233,1.915656,-1.129676,4.888587,0.618095,1.312313
2,2,-4.773330,1.836879,3.597380,-1.428516,1.683926,1.958371,0.871277,0.574266,0.687336,...,-1.744428,0.965147,-1.386724,-1.356656,-0.675435,0.012142,0.908463,-0.146876,1.643853,0.336812
3,3,-0.175352,-4.118918,-1.389481,-2.820350,-2.261878,-2.421187,0.890012,-1.181040,2.925701,...,-1.583758,1.647229,0.472433,-0.734986,0.229898,-0.701624,0.383197,-1.653472,1.196363,1.246956
4,4,0.933151,-2.467189,-0.293971,3.964782,1.173513,-2.635705,3.034213,-1.119642,-3.356119,...,-0.111398,-0.158730,0.404967,0.719784,0.176755,0.755089,-1.494247,0.432840,-0.630142,0.343732


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.073512,-3.579602,2.542294,0.829520,-1.309279,-2.129783,3.003700,-1.607696,-0.377434,0.025796,...,0.451810,0.450999,-0.897498,-0.197427,-0.556504,-1.252244,-0.218438,-0.182912,0.638235,-0.966327
1,9.926291,11.637135,-6.480039,-7.815047,-5.385470,9.829159,5.992433,-3.211267,-3.751414,8.017016,...,-0.767886,-0.963754,-0.291192,-4.728044,0.295233,1.915656,-1.129676,4.888587,0.618095,1.312313
2,-4.773330,1.836879,3.597380,-1.428516,1.683926,1.958371,0.871277,0.574266,0.687336,0.887912,...,-1.744428,0.965147,-1.386724,-1.356656,-0.675435,0.012142,0.908463,-0.146876,1.643853,0.336812
3,-0.175352,-4.118918,-1.389481,-2.820350,-2.261878,-2.421187,0.890012,-1.181040,2.925701,4.874017,...,-1.583758,1.647229,0.472433,-0.734986,0.229898,-0.701624,0.383197,-1.653472,1.196363,1.246956
4,0.933151,-2.467189,-0.293971,3.964782,1.173513,-2.635705,3.034213,-1.119642,-3.356119,-0.230266,...,-0.111398,-0.158730,0.404967,0.719784,0.176755,0.755089,-1.494247,0.432840,-0.630142,0.343732


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3697512032862768


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.509331,0.489714,0.343511,0.343511,0.370663,0.345152


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9915    0.9668    0.9790       361
           2     0.9747    0.9765    0.9756       553
           3     0.9459    0.9706    0.9581       306

    accuracy                         0.9721      1220
   macro avg     0.9707    0.9713    0.9709      1220
weighted avg     0.9724    0.9721    0.9722      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2656    0.1083    0.1538       157
           2     0.4499    0.7764    0.5697       237
           3     0.3137    0.1231    0.1768       130

    accuracy                         0.4141       524
   macro avg     0.3431    0.3359    0.3001       524
weighted avg     0.3609    0.4141    0.3476       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.509331,0.489714,0.343511,0.343511,0.370663,0.345152
1,RandomForest,0.972131,0.972131,0.972444,0.972201,0.414122,0.414122,0.360894,0.347608


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5745    0.4377    0.4969       361
           2     0.5537    0.7649    0.6424       553
           3     0.5359    0.3170    0.3984       306

    accuracy                         0.5557      1220
   macro avg     0.5547    0.5065    0.5125      1220
weighted avg     0.5554    0.5557    0.5381      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3248    0.2420    0.2774       157
           2     0.4458    0.6076    0.5143       237
           3     0.2262    0.1462    0.1776       130

    accuracy                         0.3836       524
   macro avg     0.3323    0.3319    0.3231       524
weighted avg     0.3551    0.3836    0.3598       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.514754,0.514754,0.567047,0.441298,0.412214,0.412214,0.326085,0.322346
1,RandomForest,0.972131,0.972131,0.972445,0.972097,0.429389,0.429389,0.394302,0.364478
2,Adaboost,0.555738,0.555738,0.555391,0.538108,0.383588,0.383588,0.355068,0.359766


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9692    0.8726    0.9184       361
           2     0.8379    0.9910    0.9080       553
           3     0.9793    0.7712    0.8629       306

    accuracy                         0.9008      1220
   macro avg     0.9288    0.8783    0.8964      1220
weighted avg     0.9122    0.9008    0.8998      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3173    0.2102    0.2529       157
           2     0.4327    0.6371    0.5154       237
           3     0.2535    0.1385    0.1791       130

    accuracy                         0.3855       524
   macro avg     0.3345    0.3286    0.3158       524
weighted avg     0.3537    0.3855    0.3533       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.514754,0.514754,0.567047,0.441298,0.412214,0.412214,0.326085,0.322346
1,RandomForest,0.972131,0.972131,0.972445,0.972097,0.429389,0.429389,0.394302,0.364478
2,Adaboost,0.555738,0.555738,0.555391,0.538108,0.383588,0.383588,0.355068,0.359766
3,GBM,0.900820,0.900820,0.912224,0.899769,0.385496,0.385496,0.353658,0.353291


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3829    0.3850    0.3840       361
           2     0.5665    0.3852    0.4586       553
           3     0.3222    0.5065    0.3939       306

    accuracy                         0.4156      1220
   macro avg     0.4239    0.4256    0.4121      1220
weighted avg     0.4509    0.4156    0.4203      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2500    0.2484    0.2492       157
           2     0.4207    0.2911    0.3441       237
           3     0.2059    0.3231    0.2515       130

    accuracy                         0.2863       524
   macro avg     0.2922    0.2875    0.2816       524
weighted avg     0.3163    0.2863    0.2927       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.509331,0.489714,0.343511,0.343511,0.370663,0.345152
1,RandomForest,0.972131,0.972131,0.972444,0.972201,0.414122,0.414122,0.360894,0.347608
2,SVM,0.415574,0.415574,0.450910,0.420272,0.286260,0.286260,0.316275,0.292711


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5494    0.5235    0.5362       361
           2     0.5931    0.7432    0.6597       553
           3     0.4918    0.2941    0.3681       306

    accuracy                         0.5656      1220
   macro avg     0.5448    0.5203    0.5213      1220
weighted avg     0.5548    0.5656    0.5500      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3206    0.2675    0.2917       157
           2     0.4675    0.6076    0.5284       237
           3     0.2471    0.1615    0.1953       130

    accuracy                         0.3950       524
   macro avg     0.3451    0.3455    0.3385       524
weighted avg     0.3688    0.3950    0.3749       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.514754,0.514754,0.567047,0.441298,0.412214,0.412214,0.326085,0.322346
1,RandomForest,0.972131,0.972131,0.972445,0.972097,0.429389,0.429389,0.394302,0.364478
2,Adaboost,0.555738,0.555738,0.555391,0.538108,0.383588,0.383588,0.355068,0.359766
3,GBM,0.900820,0.900820,0.912224,0.899769,0.385496,0.385496,0.353658,0.353291
4,SVM,0.464754,0.464754,0.357729,0.328356,0.442748,0.442748,0.285014,0.299834
5,KNN,0.565574,0.565574,0.554755,0.550013,0.395038,0.395038,0.368814,0.374861


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4139    0.4127    0.4133       361
           2     0.5066    0.6239    0.5592       553
           3     0.3911    0.2288    0.2887       306

    accuracy                         0.4623      1220
   macro avg     0.4372    0.4218    0.4204      1220
weighted avg     0.4502    0.4623    0.4482      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2868    0.2357    0.2587       157
           2     0.4331    0.5738    0.4936       237
           3     0.1728    0.1077    0.1327       130

    accuracy                         0.3569       524
   macro avg     0.2976    0.3057    0.2950       524
weighted avg     0.3247    0.3569    0.3337       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.514754,0.514754,0.567047,0.441298,0.412214,0.412214,0.326085,0.322346
1,RandomForest,0.972131,0.972131,0.972445,0.972097,0.429389,0.429389,0.394302,0.364478
2,Adaboost,0.555738,0.555738,0.555391,0.538108,0.383588,0.383588,0.355068,0.359766
3,GBM,0.900820,0.900820,0.912224,0.899769,0.385496,0.385496,0.353658,0.353291
4,SVM,0.464754,0.464754,0.357729,0.328356,0.442748,0.442748,0.285014,0.299834
5,KNN,0.565574,0.565574,0.554755,0.550013,0.395038,0.395038,0.368814,0.374861
6,GNB,0.462295,0.462295,0.450191,0.448156,0.356870,0.356870,0.324713,0.333718
